In [6]:
### Utility to quickly searching and accessing rules from multiple books.

#with open('', 'w') as book

class Item(object):
    def __init__(self, objType) -> None:
        # Prompts user for input upon creation.
        self.name = input("What is the object's name?")
        self.type = input("What type of object is it?")
        self.type = input("What type of object is it?")
class InfoBook(object):
    def __init__(self) -> None:
        self.edges = []
        self.hindrances = []
        self.powers = []
        self.rules = []
        self.equipment = []
    
    def addItem(self, objType):
        match objType:
            case 'edge':
                obj = Item('edge')
                self.edges.append(obj)
            case 'hindrance':
                self.hindrances.append(Item(objType))
            case 'power':
                self.powers.append(Item(objType))
            case 'rule':
                self.rules.append(Item(objType))
            case 'equipment':
                self.equipment.append(Item(objType))
                
book = InfoBook()

book.addItem('edge')

print(book.edges[0].info)

# Edge, Hindrance, Power, (Rules?), (Equipment?)
# Edge: Name, Requirements, Rules Text, Origin Book + Page Number
# Hindrance: Name, (Minor/Major), Rules Text, Origin Book + Page Number
# Power: Name, Cost, Range, Duration, Rank, Rules Text, Modifiers, (Spell Lists?), Origin Book + Page Number
# Item: Name, Rules Text (could be split into different items, weapon, armour, etc.), Origin Book + Page Number

['haha']


In [20]:
## SWADE Search Tool ##
'''
A utility to scrape text from a SWADE rulebook, extract edges,
 hinderances, powers, rules, items, and maybe statblocks. 
'''

# Get pdf
# Process pdf to extract text
# Using (regex?), extract [edges, hindrances...etc]
# Parse the extracted text
# Store each object as an SWObject


class SWObject(object):
    '''
    A general class that stores rule text for edges, hindrances, and rules.
    '''
    def __init__(self, name: str, info: str, page: int = '2', source: str = 'SWADE Core Rulebook') -> None:
        self.name = name
        self.info = info
        self.page = page
        self.source = source
    
    def show(self) -> str:
        print(f'{self.name.title()}: ({self.source} - Pg.{self.page})\n{self.info.capitalize()}')

rule = SWObject('gang up',"hey buddy, if you have multiple allies flanking the dude you're attacking, you get a bonus to fighting roles equal to the number of allies adjacent to your target. this is a test.")

rule.show()

Gang Up: (SWADE Core Rulebook - Pg.2)
Hey buddy, if you have multiple allies flanking the dude you're attacking, you get a bonus to fighting roles equal to the number of allies adjacent to your target. this is a test.


In [22]:
!pip --version
!pip install pypdf

pip 22.3.1 from c:\Users\Bardia\Desktop\git\py-practice\py-practice\.venv\lib\site-packages\pip (python 3.10)

  Using cached pypdf-3.3.0-py3-none-any.whl (239 kB)


In [ ]:
from pypdf import PdfReader
import re

reader = PdfReader('SWADE.pdf')
pages = reader.pages
#pageCount = len(reader.pages)
#print(pageCount)
#print(page.extract_text())

titlematch = r'((?:[\w]+ )+)\((M[\w \\]+)\)\n'
contentmatch = r'([\w\W]+?)\n'
#contentmatch = r'([\w]+?)\n'
query = re.compile(fr"{titlematch}{contentmatch}(?={titlematch})", flags=re.M)
#queryend = re.compile(fr"{titlematch}{contentmatch}(?!{titlematch})")

#print(page.extract_text())

pagetexts = ''
for page in pages:
    pagetexts += str(page.extract_text())
    #print(len(matches))
    #print(page.extract_text())
matches = re.findall(query, pagetexts)
for obj in matches:
    print(f'{obj[0]}({obj[1]})\n{obj[2].rstrip()}\n\n')


# (\b[^.!?]+[.!?][\s\n]){1,10}


In [57]:
from pdfminer.high_level import extract_text
import os
# os.path.join(sys.path[0], 'SWADE.pdf')
# r'C:\Users\Bardia\Desktop\git\py-practice\py-practice\SWADE.pdf'
text = extract_text(os.path.join(sys.path[0], 'SWADE.pdf'))

titlematch = r'((?:[\w]+ )+)\((Minor|Major|Minor or Major)\)\n'
# old titlematch = r'((?:[\w]+ )+)\((M[\w \\]+)\)\n'
contentmatch = r'([\w\W]+?)\n'
# ignore contentmatch = r'([\w]+?)\n'
query = re.compile(fr"""
    ((?:[\w]+\s)+)\((Minor|Major|Minor or Major)\)\n
    ([\w\W]+?)\n
    (?=(?:(?:[\w]+\s)+)\((?:Minor|Major|Minor\sor\sMajor)\)\n)""", flags=re.M | re.I | re.X)
#queryend = re.compile(fr"{titlematch}{contentmatch}(?!{titlematch})")

#print(page.extract_text())


matches = re.findall(query, text)
for obj in matches:
    print(f'{obj[0]}({obj[1]})\n{obj[2].rstrip()}\n------------\n')


All Thumbs (Minor)
Due  to  upbringing,  lack  of  exposure,  or 
pure  bad  luck,  some  individuals  are  “all 
thumbs”  when  it  comes  to  mechanical  or 
technological devices.

All Thumbs inflicts a −2 penalty when using 
mechanical or electrical devices. If he rolls a 
Critical  Failure  while  using  such  a  device 
(and it doesn’t already have a built-in effect), 
it’s broken. If the GM feels it’s appropriate, it 
can be fixed with a Repair roll and 1d6 hours.
------------

Anemic (Minor)
Anemic characters are particularly susceptible 
to  sickness,  disease,  environmental  effects, 
and fatigue. They subtract 2 from Vigor rolls 
made to resist Fatigue (see Hazards, starting 
on page 125).
------------

Arrogant (Major)
Your  hero  doesn’t  think  he’s  the  best — he 
knows  he  is.  Whether  it’s  swordsmanship, 
kung fu, or painting, few compare to his skills 
and he flaunts it every chance he gets.

Winning  isn’t  enough  for  this  champion. 
He  wants  to  dominate  

In [54]:
import pdfplumber, os, re
import statistics

with pdfplumber.open(os.path.join(sys.path[0], 'SWADE.pdf')) as pdf:    
    # find the top bounding box of the page number
    pageNumTop = max(pdf.pages[12].extract_words(), key=lambda p: p['top'])['top']
    pageWidth = pdf.pages[12].width
    pageHeight = pdf.pages[12].height

    words = pdf.pages[29].extract_words()
        
    # Create a set of all general font sizes on the page.
    fontSizes = set(round(word['bottom']-word['top']) for word in words)

    #print(fontSizes)

    # Sort all the words on the page into a tiered list, split by font size
    wordsByFont = []
    for fontSize in fontSizes:
        wordsByFont.append([fontSize])
        for word in words:
            if round(word['bottom']-word['top']) == fontSize:
                wordsByFont[-1].append(word)
    
    print(wordsByFont[3])


    

    #for word in words: xcoords.append((word['text'], word['bottom']-word['top']))
    
    #print(statistics.quantiles((word['bottom']-word['top'] for word in words)))

    #print(median(xcoords))

    #print(xcoords)

    # pdf.pages[15].search(r'([\w\W]+?)\n')


[14, {'text': 'Tongue-Tied', 'x0': 36.0, 'x1': 89.9016, 'top': 271.424, 'doctop': 21431.377, 'bottom': 284.96, 'upright': True, 'direction': 1}]
